In [1]:
import sys
import pandas as pd

sys.path.append('F:/repo/gpsbeam')

import onnxruntime as ort
import numpy as np
import torch
import time
import timeit
import concurrent
import matplotlib.pyplot as plt
import gc

from src.config.data_config import DataConfig
from src.data.dataprep import DataPrep
from src.data.base_datatotensor import BaseDataToTensor
from src.preprocessor.gpsprep import GpsPrep

from src.config.data_config import DataConfig
from src.config.cnn_ed_rnn_model_config import ModelConfig
from src.config.experiment_config import ExperimentConfig
from src.modelprep import ModelPrep

# Data Set Preparation

In [2]:
data_config_obj = DataConfig(train_val_test_split_frac=[0.65, 0.15, 0.2], 
                             splitting_method='adjusted',
                             num_classes=32,
                             scenario_num=23)

dataprep_obj = DataPrep(data_config_obj)

dataprep_obj.get_train_val_test_dataset()

2025-05-20 15:26:06.033 | INFO     | src.data.dataprep:get_train_val_test_dataset:296 - 
Dataset is LOADED from f:/repo/gpsbeam\data/processed/Scenario23/dset_scenario23_seednum42_train0.65_test0.2_portion100_beam32_splitting_method_adjusted.hkl
2025-05-20 15:26:06.033 | INFO     | src.data.dataprep:get_train_val_test_dataset:311 - 
                    RAW DATASET INFO
                    ------------------------------
                    Scenario Num                                    : 23,
                    Splitting Method                                : adjusted
                    Portion Percentage                              : 100
                    Training                                        : 7387 samples [64.87%]
                    Validation                                      : 1694 samples [14.88%]
                    Testing                                         : 2306 samples [20.25%]
                    Total                                           : 1138

# Load Trained Pytorch & ONNX Models

In [3]:
# Load the ONNX model
torch_fname = r'F:\repo\gpsbeam\data\experiment_result\00_test_drone_cnn_ed_rnn_experiment_20052025_151107\05-20-2025_15_11_29\dl_generated\model_checkpoint\arch_cnn-ed-gru-model_nclass_32_.pth'
onnx_fname = r'F:\repo\gpsbeam\data\experiment_result\00_test_drone_cnn_ed_rnn_experiment_20052025_151107\05-20-2025_15_11_29\dl_generated\model_checkpoint\arch_cnn-ed-gru-model_nclass_32_.onnx'
ort_session = ort.InferenceSession(onnx_fname,
                                   providers=["CPUExecutionProvider"])



In [8]:
model_config_obj=ModelConfig(model_arch_name='cnn-ed-gru-model',
                                       train_epoch=20,
                                       train_batch_size=8,
                                       test_batch_size=1024,
                                       use_early_stopping=False,
                                       device='cpu',
                                       model_input_column_list=['unit2to1_vector', 'unit2_height_log'],
                                       zero_pad_nonconsecutive=True,
                                       ends_input_with_out_len_zeros=False,
                                       seq_len=8,
                                       out_len=3,
                                       cnn_channels=[128, 128],
                                       rnn_num_layers=1,
                                       rnn_hidden_size=128,
                                       mlp_layer_sizes=[64],
                                       rnn_dropout=0, # fix
                                       cnn_dropout=0, # fix
                                       adam_weight_decay=0, # fix
                                       loss_func_name='cross-entropy-loss',
                                       adam_learning_rate=5e-4, # fix
                                       adam_opt_milestone_list=[12, 18],
                                       num_classes=32
                                       )

exp_config_obj=ExperimentConfig(
                  exp_folder_name='00_test_drone_cnn_ed_rnn_experiment',
                  exp_dict={
                      "model_arch_name": ['cnn-ed-gru-model'
                                        ],
                       "num_classes": [32]
                  })

modelprep_obj = ModelPrep(experiment_config=exp_config_obj,
                                    data_config=data_config_obj,
                                    model_config=model_config_obj,
                                    dataset_dict=dataprep_obj.dataset_dict)

modelprep_obj._setup_model()
modelprep_obj.model.load_state_dict(torch.load(torch_fname))

test_dataset = modelprep_obj._create_tensor_dataset(data_dict=dataprep_obj.dataset_dict['test'])

2025-05-20 15:28:10.594 | INFO     | src.modelprep:_setup_paths:51 - model_recap_dir Name: f:/repo/gpsbeam\data/experiment_result/00_test_drone_cnn_ed_rnn_experiment_20052025_152810/model_recap
2025-05-20 15:28:10.611 | DEBUG    | src.preprocessor.gpsprep:__init__:8 - GpsPrep Initialized.


# Show an Example of Model Input

In [9]:
test_dataset[0]

{'input_from_scenario': tensor([ 0,  0,  0,  0, 23, 23, 23, 23]),
 'input_seq_idx': tensor([0, 0, 0, 0, 1, 1, 1, 1]),
 'input_speed': tensor([0.0000, 0.0000, 0.0000, 0.0000, 7.6056, 6.4871, 5.8160, 4.6976]),
 'input_height': tensor([  0.0000,   0.0000,   0.0000,   0.0000, 103.6745, 103.6745, 103.6745,
         103.6745]),
 'input_value': tensor([[ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [-0.3218, -0.7464,  0.5825,  4.6509],
         [-0.3247, -0.7451,  0.5826,  4.6509],
         [-0.3270, -0.7440,  0.5827,  4.6509],
         [-0.3297, -0.7427,  0.5829,  4.6509]]),
 'input_beam_idx': tensor([ 0,  0,  0,  0, 14, 14, 14, 14]),
 'input_pitch': tensor([ 0.0000,  0.0000,  0.0000,  0.0000, -3.4000, -3.3000, -2.8000, -1.4000]),
 'input_roll': tensor([ 0.0000,  0.0000,  0.0000,  0.0000, 13.1000, 12.6000, 12.2000,  9.5000]),
 'label': tensor([14, 14, 1

# Inference Time Test


In [10]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# Apply softmax to the raw output to get probabilities
def softmax(x):
    """Compute softmax values for each set of scores in x."""
    e_x = np.exp(x - np.max(x))  # Subtract max for numerical stability
    return e_x / e_x.sum()

def get_top_5_beams_inference_time(sample, num_iteration):
    """
    Get top 5 beam predictions and measure inference time for both ONNX and PyTorch models
    
    Args:
        sample: Input tensor sample
        
    Returns:
        tuple: (onnx_indices, torch_indices, onnx_time_ms, torch_time_ms)
    """
    def onnx_inference():
        # Reshape input for ONNX model (add batch dimension)
        input_reshaped = sample.unsqueeze(0)
        
        # Run ONNX inference
        ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(input_reshaped)}
        ort_outs = ort_session.run(['output'], ort_inputs)
        
        # Get probabilities from logits
        all_pred_steps = ort_outs[0][0] # 4 prediction step
        
        # Apply softmax to all predictions at once
        probabilities_all = softmax(all_pred_steps)
        # Get top 5 indices for all predictions at once
        top_5_beams_all = np.argsort(probabilities_all, axis=1)[:, ::-1][:, :5]
        # Convert to list of arrays
        # all_pred_sorted = [top_5_beams_all[i] for i in range(len(top_5_beams_all))]
        return top_5_beams_all

    def torch_inference():
        # Reshape input for PyTorch model
        input_reshaped = sample.unsqueeze(0)
        
        # Run PyTorch inference
        with torch.no_grad():
            outputs= modelprep_obj.model(input_reshaped)
            all_pred_steps = outputs[0][0] # 4 prediction step
            
            # Convert all predictions to numpy at once
            logits_all = all_pred_steps.numpy()
            # Apply softmax to all predictions at once
            probabilities_all = softmax(logits_all)
            # Get top 5 indices for all predictions at once
            top_5_beams_all = np.argsort(probabilities_all, axis=1)[:, ::-1][:, :5]
            # Add each prediction's top 5 beams to the result list
            # all_pred_sorted = [top_5_beams_all[i] for i in range(len(top_5_beams_all))]
            return top_5_beams_all
    
    # Measure ONNX inference time
    onnx_time_ms = timeit.timeit(onnx_inference, number=num_iteration) * 1000 / num_iteration
    
    # Measure PyTorch inference time  
    torch_time_ms = timeit.timeit(torch_inference, number=num_iteration) * 1000 / num_iteration
    
    # Run once more to get actual indices
    onnx_indices = onnx_inference()
    torch_indices = torch_inference()
    
    return onnx_indices, torch_indices, onnx_time_ms, torch_time_ms

# Test the function
num_iteration = 10_000
sample = test_dataset[0]['input_value']
onnx_indices, torch_indices, onnx_time_ms, torch_time_ms = get_top_5_beams_inference_time(sample, num_iteration)
print(f"ONNX inference time: {onnx_time_ms:.4f} ms")
print(f"PyTorch inference time: {torch_time_ms:.4f} ms")
print(f"ONNX is {torch_time_ms/onnx_time_ms:.2f}x faster than PyTorch")
print(f"ONNX top 5 beam indices: {onnx_indices}")
print(f"PyTorch top 5 beam indices: {torch_indices}")

ONNX inference time: 0.1992 ms
PyTorch inference time: 1.1623 ms
ONNX is 5.83x faster than PyTorch
ONNX top 5 beam indices: [[14 15 16 13 18]
 [14 15 16 13 18]
 [14 15 13 16 18]
 [14 15 16 13 18]]
PyTorch top 5 beam indices: [[14 15 13 16 18]
 [14 15 13 16 19]
 [14 15 13 16 19]
 [14 15 13 16 19]]


# Input Output Checking

In [11]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# Apply softmax to the raw output to get probabilities
def softmax(x):
    """Compute softmax values for each set of scores in x."""
    e_x = np.exp(x - np.max(x))  # Subtract max for numerical stability
    return e_x / e_x.sum()

# Get the predicted class (beam index with highest probability)
def get_top_beams_until_threshold(probabilities, threshold=0.8):
    """
    Get the top beam indices until their cumulative probability reaches the threshold.
    
    Args:
        probabilities: Array of probabilities for each beam
        threshold: Probability threshold (default: 0.8)
        
    Returns:
        List of (beam_index, probability) tuples and the total number of beams needed
    """
    # Sort probabilities in descending order and get corresponding indices
    sorted_indices = np.argsort(probabilities)[::-1]
    sorted_probs = probabilities[sorted_indices]
    
    # Find how many beams we need to reach the threshold
    cumulative_sum = 0
    selected_beams = []
    
    for i, (idx, prob) in enumerate(zip(sorted_indices, sorted_probs)):
        cumulative_sum += prob
        selected_beams.append((idx, prob))
        
        if cumulative_sum > threshold:
            break
    
    return selected_beams, len(selected_beams)

# The ONNX model expects input with rank 3 (batch_size, sequence_length, features)
# Reshape the input tensor to add batch dimension and sequence dimension
sample = test_dataset[0]['input_value']
print(sample)
print(f"sample.shape: {sample.shape}")
input_reshaped = sample.unsqueeze(0)  # Add batch dimension
# input_reshaped = input_reshaped.unsqueeze(0)  # Add sequence dimension
print(f"input_reshaped.shape: {input_reshaped.shape}")
print(input_reshaped)

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(input_reshaped)}
ort_outs, h = ort_session.run(None, ort_inputs)


all_pred_steps = ort_outs[0] # 4 prediction step
logits = all_pred_steps[0] # 1st prediction step

# Apply softmax to convert logits to probabilities
probabilities = softmax(logits)

print(f"Sum of probabilities: {np.sum(probabilities)}")  # Should be close to 1.0

# Get the predicted beam with highest probability
predicted_beam = np.argmax(probabilities)
print(f"Predicted beam index: {predicted_beam}")
print(f"Probability of predicted beam: {probabilities[predicted_beam]:.6f}")

# Get top beams until threshold
threshold = 0.8
top_beams, num_beams_needed = get_top_beams_until_threshold(probabilities, threshold)

print(f"\nTop beams until {threshold*100}% confidence:")
for beam_idx, prob in top_beams:
    print(f"Beam {beam_idx}: {prob:.6f}")
print(f"Number of beams needed to reach {threshold*100}% confidence: {num_beams_needed}")


tensor([[ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [-0.3218, -0.7464,  0.5825,  4.6509],
        [-0.3247, -0.7451,  0.5826,  4.6509],
        [-0.3270, -0.7440,  0.5827,  4.6509],
        [-0.3297, -0.7427,  0.5829,  4.6509]])
sample.shape: torch.Size([8, 4])
input_reshaped.shape: torch.Size([1, 8, 4])
tensor([[[ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [-0.3218, -0.7464,  0.5825,  4.6509],
         [-0.3247, -0.7451,  0.5826,  4.6509],
         [-0.3270, -0.7440,  0.5827,  4.6509],
         [-0.3297, -0.7427,  0.5829,  4.6509]]])
Sum of probabilities: 1.0
Predicted beam index: 14
Probability of predicted beam: 0.538693

Top beams until 80.0% confidence:
Beam 14: 0.538693
Beam 15: 0.307830
Number of beam